<div align="right">(18-Mar-2022)</div>
<h1>Caclulate a Set of Descriptors and Create Plots</h1>
     
* Create per-class and overview plots

In [1]:
%reload_ext autoreload
%autoreload 2
def warn(*args, **kwargs):
    pass  # to silence scikit-learn warnings

import warnings
warnings.filterwarnings('ignore')
warnings.warn = warn

# Type hints
from typing import Iterable, List, Set, Dict, Union, Optional

import os,gc

import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn.decomposition import PCA
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
# from scipy.stats import median_absolute_deviation as mad

from rdkit import DataStructs
from rdkit.Chem import AllChem as Chem, QED
from rdkit.Chem import Descriptors as Desc
from rdkit.Chem import rdMolDescriptors as rdMolDesc
from rdkit.Chem import Fragments
# from rdkit.Chem import Draw
# from rdkit.Chem.Draw import IPythonConsole

# from mol_frame import mol_frame as mf
# from cellpainting3 import processing as cpp, tools as cpt
from Contrib.NP_Score import npscorer

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
tqdm.pandas()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Project-local Imports
from jupy_tools import plt_style, pca
from jupy_tools import utils as u, mol_view as mv
from jupy_tools.utils import info

## Functions

## Configuration

In [2]:
config = {
    "dataset": "smiles_all",
    "name": "SmoAnta",
    "id_col": "Cpd_Id",  # this key is optional
    "extra_columns": [],
    # when the data set contains internal Compound_Ids, these will be used for merging thestructures. 
    # Otherwise a "Smiles" column is expected:
    "has_cpd_ids": False,  
    "per_class_plots": True,
    "per_class_order_asc": False,
    "per_class_colors": ["red", "blue"]
}

In [3]:

DATA_EN = "enamine_adv_full"
DATA_DB = "drugbank_5.1.8_appr_inv_full"
DATA_NP = "chembl_30_np_full_nocanon_deglyco"

descriptors = {
    "NP_Like": lambda x: round(score_np(x), 2), 
    "QED": lambda x: round(QED.default(x), 3),
    "NumHA": lambda x: x.GetNumAtoms(),
    "MW": lambda x: round(Desc.ExactMolWt(x), 2),
    "NumRings": rdMolDesc.CalcNumRings,
    "NumRingsArom": rdMolDesc.CalcNumAromaticRings,
    "NumRingsAli": rdMolDesc.CalcNumAliphaticRings,
    "NumHDon": rdMolDesc.CalcNumLipinskiHBD,
    "NumHAcc": rdMolDesc.CalcNumLipinskiHBA,
    "LogP": lambda x: round(Desc.MolLogP(x), 2),
    "TPSA": lambda x: round(rdMolDesc.CalcTPSA(x), 2),
    "NumRotBd": rdMolDesc.CalcNumRotatableBonds,
    "NumAtOx": lambda x: len(
        [a for a in x.GetAtoms() if a.GetAtomicNum() == 8]
    ),
    "NumAtN": lambda x: len(
        [a for a in x.GetAtoms() if a.GetAtomicNum() == 7]
    ),
    "NumAtHal": Fragments.fr_halogen,
    "NumAtBridgehead": rdMolDesc.CalcNumBridgeheadAtoms,
    "FCsp3": lambda x: round(rdMolDesc.CalcFractionCSP3(x), 3), 
}

cmap = {"Enamine": "#1f77b4", "DrugBank": "#ff7f0e", "ChEMBL NP": "#2ca02c", config["name"]: "#d62728"}
contour_cmap = {"Enamine": "#1f77b4", "DrugBank": "#ff7f0e", "ChEMBL NP": "#2ca02c", config["name"]: "#d62728"}
selected_descriptors = ["NP_Like", "QED", "FCsp3"]
xlims = {"NP_Like": (-5, 5), "QED": (0, 1), "FCsp3": (0, 1)}
widths = {"NP_Like": 10, "QED": 10, "FCsp3": 10}

fscore = npscorer.readNPModel()
def score_np(mol):
    return npscorer.scoreMol(mol, fscore)

os.makedirs(f'output/{config["dataset"]}_desc', exist_ok=True)

reading NP model ...
model in


## Loading Data Sets

In [4]:
df_int = u.read_tsv(f"input/{config['dataset']}.tsv")

if config["has_cpd_ids"]:
    # Merge structure Smiles by Compound_Id
    config["id_col"] = "Compound_Id"
    df_comas = u.read_tsv("/home/pahl/comas/share/comas_smiles.tsv")
    num_cpds_1 = len(df_int)
    df_int = pd.merge(df_int, df_comas, on="Compound_Id", how="left")
    num_cpds_2 = len(df_int)
    assert num_cpds_1 == num_cpds_2, f"Only {num_cpds_2} out of {num_cpds_2} compounds were found in the COMAS database."
if config.get("has_salts", False):
    print("Standardizing structures...")
    num_cpds_1 = len(df_int)
    df_int = u.calc_from_smiles(df_int, "Smiles_Stand", u.standardize_mol)
    df_int = u.remove_nans(df_int, "Smiles_Stand")
    num_cpds_2 = len(df_int)
    assert num_cpds_1 == num_cpds_2, f"Only {num_cpds_2} out of {num_cpds_1} compounds could be standardized."
    df_int = df_int.drop("Smiles", axis=1)
    df_int = df_int.rename(columns={'Smiles_Stand': 'Smiles'})
if "id_col" not in config:
    config["id_col"] = "Cpd_Id"
    df_int["Cpd_Id"] = df_int.index + 1

# The reference data sets already contain the Descriptor data
# Generated by the separate notebook `02_desc_precalc_datasets`
df_en = u.read_tsv(f"input/{DATA_EN}_sample_50k_desc.tsv")
df_db = u.read_tsv(f"input/{DATA_DB}_desc.tsv")
df_np = u.read_tsv(f"input/{DATA_NP}_desc.tsv")

datasets = {"Enamine": df_en, "DrugBank": df_db, "ChEMBL NP": df_np}
id_cols = {"Enamine": "idnumber", "DrugBank": "DRUGBANK_ID", "ChEMBL NP": "chembl_id", config["name"]: config["id_col"]}

read_tsv                           : [     616 /   6 ] ( Cpd_Id, Name, InChIKey, Smiles, DataSet, CpdClass )
read_tsv                           : [   50000 /  28 ] 
read_tsv                           : [    4866 /  21 ] 
read_tsv                           : [   45679 /  20 ] 


## Write Structure Overview File

In [5]:
mv.write_mol_grid(
    df_int, title=config["name"], id_col="Cpd_Id", fn=f"output/{config['dataset']}_desc/mol_grid.html",
    truncate=20
)

  0%|          | 0/616 [00:00<?, ?it/s]

drop_cols                          : [     616 /   6 ]  1 columns removed. ( Cpd_Id, Name, InChIKey, DataSet, CpdClass, Mol )


## Adding Descriptors to Internal Data

In [6]:
for desc in descriptors:
    print(f"{desc:15}: ")
    df_int = u.calc_from_smiles(df_int, desc, descriptors[desc])
datasets[config["name"]] = df_int
u.write_tsv(df_int, f"output/{config['dataset']}_desc/descriptors.tsv")

NP_Like        : 


  0%|          | 0/616 [00:00<?, ?it/s]

calc_from_smiles                   : [     616 /   7 ]    0 rows removed because of nans. ( Cpd_Id, Name, InChIKey, Smiles, DataSet, CpdClass, NP_Like )
QED            : 


  0%|          | 0/616 [00:00<?, ?it/s]

calc_from_smiles                   : [     616 /   8 ]    0 rows removed because of nans. ( Cpd_Id, Name, InChIKey, Smiles, DataSet, CpdClass, NP_Like, QED )
NumHA          : 


...

calc_from_smiles                   : [     616 /  21 ]    0 rows removed because of nans. 
NumAtBridgehead: 


  0%|          | 0/616 [00:00<?, ?it/s]

calc_from_smiles                   : [     616 /  22 ]    0 rows removed because of nans. 
FCsp3          : 


  0%|          | 0/616 [00:00<?, ?it/s]

calc_from_smiles                   : [     616 /  23 ]    0 rows removed because of nans. 


## Generate One Data Set for Plotting

In [7]:
tmp_list = []
for ds in datasets:
    tmp = datasets[ds].copy()
    tmp["DataSet"] = ds
    tmp = tmp.rename(columns={id_cols[ds]: "Cpd_Id"})
    tmp_list.append(tmp)

cols = ["Cpd_Id", "DataSet"]
cols.extend(config["extra_columns"])
cols.extend(descriptors.keys())
if config["per_class_plots"]:
    cols.append("CpdClass")
df_all = pd.concat(tmp_list)[cols]
df_all = df_all.reset_index(drop=True)
del tmp_list

## Analysis
### Overview Plots NP-Likeness, QED
### ECDF

In [8]:
for desc in selected_descriptors:
    plt.figure(figsize=(widths[desc], 7))
    sns.ecdfplot(data=df_all, x=desc, stat="proportion", hue="DataSet", palette=cmap)
    plt.xlim(xlims[desc]);
    # plt.legend();
    plt.title(f"Distribution of {desc}")
    plt.savefig(f"output/{config['dataset']}_desc/ecdf_overview_{desc}.png");
    plt.savefig(f"output/{config['dataset']}_desc/ecdf_overview_{desc}.svg");
    plt.clf()
    plt.close()
    gc.collect()

<Figure size 720x504 with 0 Axes>

<AxesSubplot:xlabel='NP_Like', ylabel='Proportion'>

(-5.0, 5.0)

Text(0.5, 1.0, 'Distribution of NP_Like')

531

...

<Figure size 720x504 with 0 Axes>

<AxesSubplot:xlabel='FCsp3', ylabel='Proportion'>

(0.0, 1.0)

Text(0.5, 1.0, 'Distribution of FCsp3')

1760

### Per-Class Plots NP-Likeness, QED

In [9]:
if config["per_class_plots"]:
    cpd_classes = sorted(df_int["CpdClass"].unique())
    print(cpd_classes)

    for desc in selected_descriptors:
        plt.figure(figsize=(widths[desc], 7))
        sns.ecdfplot(data=df_int, x=desc, stat="proportion", hue="CpdClass")
        plt.xlim(xlims[desc]);
        # plt.legend();
        plt.title(f"Distribution of {desc}")
        plt.savefig(f"output/{config['dataset']}_desc/ecdf_perclass_{desc}.png");
        plt.savefig(f"output/{config['dataset']}_desc/ecdf_perclass_{desc}.svg");
        plt.clf()
        plt.close()
        gc.collect()

['PNP', 'SmoAnta']


<Figure size 720x504 with 0 Axes>

<AxesSubplot:xlabel='NP_Like', ylabel='Proportion'>

(-5.0, 5.0)

Text(0.5, 1.0, 'Distribution of NP_Like')

...

<Figure size 720x504 with 0 Axes>

<AxesSubplot:xlabel='FCsp3', ylabel='Proportion'>

(0.0, 1.0)

Text(0.5, 1.0, 'Distribution of FCsp3')

1760